In [1]:
import glob
from RHESSysLandCover import *

In [2]:
dat = pd.read_pickle('./data/experiment_forcing_params_dynamic.pcl')
dat['idx'] = np.arange(1,len(dat)+1)

In [3]:
dat.tail(12)

,P,T,decade,krumlai,krumweight,name,scenarios,track,year,idx
0,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,1,PGW_MP,2000,115
1,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,2,PGW_MP,2000,116
2,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,3,PGW_MP,2000,117
3,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,1,PGW_MP,2000,118
4,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,2,PGW_MP,2000,119
5,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,3,PGW_MP,2000,120
0,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,1,PGW_LP,2000,121
1,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,2,PGW_LP,2000,122
2,-0.075,2.0,5.0,NaN,NaN,CLC_LP_FT,5,PGW_LP,2000,123
3,-0.150,4.0,10.0,NaN,NaN,CLC_LP_FT,1,PGW_LP,2000,124


In [4]:
dat.shape

(126, 10)

In [5]:
strt = '1993-10-1'
nd = '2013-09-30'

In [6]:
df = pd.DataFrame({'file':glob.glob('/RHESSys/Como/out/exp/opt_35_312/*_basin.daily')})

In [8]:
obj = process(df.file[0])

In [12]:
obj.year

2000

In [11]:
obj.annual.R_P

0.10981007887736775

In [20]:
res = df.apply(process,axis=1)

In [21]:
names = ['track','scenario','year','decade','name','idx','R_P','LAI','R_Pmin','R_Pmax','Q','L','P','SWE','ET',
         'leakage','streamflow','et','trans','snow_sub','snowfall','precip','evap','leakage','root_storage','pet',
         'swe','snowmelt','annualQ','annualP']
for name,val in zip(names,zip(*res)):
    df[name] = val

In [22]:
df.sort_values('idx',inplace=True)

In [23]:
# removed b/c full run with all the climate futures do not have the correct index numbers, but those numbers are not important
# add the columns from the experiment parameter file
df['krumlai'] = dat['krumlai'].as_matrix()
df['krumweight'] = dat['krumweight'].as_matrix()

In [24]:
del df['file']

In [25]:
df.reset_index(inplace=True)

In [26]:
df.to_pickle('./data/experiment_results_v6.pcl')

In [59]:
! ls -ltrh ./data/experiment_results_*.pcl

-rwx------  1 barnhatb  staff   8.7M Oct 17 14:36 ./data/experiment_results_v2.pcl
-rwx------  1 barnhatb  staff    17M Oct 20 08:50 ./data/experiment_results_v3.pcl
-rwx------  1 barnhatb  staff    49M Oct 25 17:25 ./data/experiment_results_v4.pcl
-rwx------  1 barnhatb  staff    70M Nov  7 17:04 ./data/experiment_results_v5.pcl


In [32]:
df.year.unique()

array([2000, 2010, 2020, 2030, 2040, 2050, 2060, 2070, 2080, 2090, 2100])